In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

# DataMap examples

`DataMaps` are configurable objects with the purpose of translating PV values to simulation inputs. They do *not* retain any values. Rather, they provide methods `.to_tao(pvdata)`, `.to_bmad(pvdata)` that will produce input for Tao and Bmad, respectively, from a dict-like `pvdata` object with the actual PV names and values. 

## PVDATA

Get some actual data that we will use to map

In [ ]:
import json
import os

In [ ]:
PVDATA = json.load(open('data/PVDATA-2021-04-21T08:10:25.000000-07:00.json'))
len(PVDATA)

# Tabular

Often PVs have a simple linear mapping to simulation inputs. The `TabularDataMap` helps with this

In [ ]:
from lcls_live.datamaps.tabular import TabularDataMap
from lcls_live import data_dir

import pandas as pd
import dataclasses 

In [ ]:
# Make some tabular data
dat = [
    
    {'name': 'BC1_offset',
     'pvname':'BMLN:LI21:235:MOTR',  # mm
     'bmad_factor': 0.001,
     'bmad_name': 'O_BC1_OFFSET',
     'bmad_attribute': 'offset'
    },
    
    {'name': 'BC2_offset',
     'pvname':'BMLN:LI24:805:MOTR', # mm
     'bmad_factor': 0.001,
     'bmad_name': 'O_BC2_OFFSET',
     'bmad_attribute': 'offset'
    },
    
    {
    'name': 'L1_phase',
    'description': 'Controls the L1 phase, which is the single klystron L21_1. We will disable this for now, because the KlystronDataMap handles the phase directly.',
    'pvname': 'ACCL:LI21:1:L1S_S_PV',
    'bmad_name':'O_L1',
    'bmad_factor': 0,  # We'll disable this for now. The Klystron handles it. 
    'bmad_attribute':'phase_deg'
    
},    
    
    {
    'name': 'L2_phase',
    'pvname': 'ACCL:LI22:1:PDES',
    'bmad_name':'O_L2',
    'bmad_factor': 1,
    'bmad_attribute':'phase_deg'
    
},
    {
    'name': 'L3_phase',
    'pvname': 'ACCL:LI25:1:PDES',
    'bmad_name':'O_L3',
    'bmad_attribute':'phase_deg',
    'bmad_offset': 0
    
},
]






#Note that there are sone NaNs here. That's okay.
df = pd.DataFrame(dat)
df

In [ ]:
# Make the DataMap object, identifying the columns to be used
DM = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor', offset='bmad_offset')

DM.pvlist

In [ ]:
# Process the data for Bmad commands
DM.as_bmad(PVDATA)

In [ ]:
# or Tao commands
DM.as_tao(PVDATA)

In [ ]:
# Save, and reload
JSON_OUT = os.path.join(data_dir, 'cu/linac_TabularDataMap.json')

DM.to_json(JSON_OUT)

DM2 = TabularDataMap.from_json(open(JSON_OUT).read())
DM2.data

## from CSV

In [ ]:
# Read a previously made csv file. This has slightly different columns
df2 = pd.read_csv('../lcls_live/data/cu_hxr/quad_mapping.csv')[0:10]
df2.columns

In [ ]:
df2['pvname'] = df2['device_name']+':'+df2['attribute']

DM2 = TabularDataMap(df2, pvname='pvname', element='bmad_ele_name', attribute='bmad_attribute', factor='bmad_factor', offset='')
DM2.pvlist

In [ ]:
# Here these aren't in our PVDATA
DM2.as_tao(PVDATA)

In [ ]:
# We could  check beforehand:
missing = [name for name in DM2.pvlist if name not in PVDATA]
len(missing)

## Quads from Pytao

In [ ]:
from pytao import Tao
tao = Tao('-init $LCLS_LATTICE/bmad/models/cu_hxr/tao.init -slice OTR2:END -noplot')

In [ ]:
def quad_pvinfo(tao, ele):
    """
    Returns dict of PV information for use in a DataMap
    """
    head = tao.ele_head(ele)
    attrs = tao.ele_gen_attribs(ele)
    device = head['alias']
    
    d = {}
    d['bmad_name'] = ele
    d['pvname_rbv'] = device+':BACT'
    d['pvname'] = device+':BDES'    
    d['bmad_factor'] = -1/attrs['L']/10
    d['bmad_attribute'] = 'b1_gradient'
    return d

quad_pvinfo(tao, 'QM01')

In [ ]:
quad_names = tao.lat_list('quad::*', 'ele.name', flags='-no_slaves')

dfq = pd.DataFrame([quad_pvinfo(tao, ele) for ele in quad_names])
dfq

In [ ]:
QUAD_DATAMAP = TabularDataMap(dfq, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute', factor='bmad_factor')

In [ ]:
JSONFILE = os.path.join(data_dir, 'cu_hxr/quad_TabularDataMap.json')
QUAD_DATAMAP.to_json(JSONFILE)

## Measurements for Tao

In [ ]:
# The syntax is flexible enough to use for getting measurements for Tao
ENERGY_MEAS = [
    {
    'name': 'L1_energy',
    'pvname': 'BEND:LI21:231:EDES', # or EDES
    'tao_datum': 'BC1.energy[1]',        
    'factor': 1e9
    },
    {
    'name': 'L2_energy',
    'pvname': 'BEND:LI24:790:EDES', # or EDES
    'tao_datum': 'BC2.energy[1]',
    'factor': 1e9
    },
    {
    'name': 'L3_HXR_energy',
    'pvname': 'BEND:DMPH:400:EDES', # or EDES
    'tao_datum': 'L3.energy[2]',
    'factor': 1e9
    }
   #{
   #'name': 'L3_SXR_energy',
   #'pvname': 'BEND:DMPS:400:EDES', # or EDES
   #'factor': 1e9
   #},       
]
df = pd.DataFrame(ENERGY_MEAS)
DM = TabularDataMap(df, pvname='pvname', element='tao_datum', factor='factor',
                   tao_format = 'set data {element}|meas  = {value}',
                   bmad_format = '! No equivalent Bmad format for: set data {element}|meas  = {value}'
                   )
DM.as_tao(PVDATA)

In [ ]:
# and this will produce
DM.as_bmad(PVDATA)

In [ ]:
# Save
JSON_OUT = os.path.join(data_dir, 'cu_hxr/tao_energy_measurements_TabularDataMap.json')
DM.to_json(JSON_OUT)

## Subboosters

In [ ]:
from lcls_live.datamaps.klystron import subbooster_pvinfo, SUBBOOSTER_SECTORS

In [ ]:
SUBBOOSTERS = []
for sector in SUBBOOSTER_SECTORS:
    
    dat = subbooster_pvinfo(sector) 
    dat['bmad_name'] = f'SBST_{sector}'
    dat['bmad_attribute'] = 'phase_deg'
    SUBBOOSTERS.append(dat)
df = pd.DataFrame(SUBBOOSTERS)    
df  

In [ ]:
# Make the DataMap object, identifying the columns to be used
DM = TabularDataMap(df, pvname='phase_pvname', element='bmad_name', attribute='bmad_attribute')

JSON_OUT = os.path.join(data_dir, 'cu/subboosters_TabularDataMap.json')

DM.to_json(JSON_OUT)

    
# Load:
#TabularDataMap.from_json(JSON_OUT)  

## Beginning at OTR2

In [ ]:
OTR2_MEAS = [
    {
    'name': 'OTR2_beta_x_meas',
    'pvname': 'OTRS:IN20:571:BETA_X', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'beta_a'
    },
    {
    'name': 'OTR2_beta_y_meas',
    'pvname': 'OTRS:IN20:571:BETA_Y', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'beta_b'
    },
    {
    'name': 'OTR2_alpha_x_meas',
    'pvname': 'OTRS:IN20:571:ALPHA_X', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'alpha_a'
    },
    {
    'name': 'OTR2_alpha_y_meas',
    'pvname': 'OTRS:IN20:571:ALPHA_Y', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'alpha_b'
    },
    
    # Some others:
    #'OTRS:IN20:571:EMITN_X',
    #'OTRS:IN20:571:EMITN_Y',
    #'OTRS:IN20:571:EMIT_TIME'
    
]
df = pd.DataFrame(OTR2_MEAS)

## OTR2_BEGINNING = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute')
OTR2_BEGINNING.data

In [ ]:
JSON_OUT = os.path.join(data_dir, 'cu/beginning_OTR2_TabularDataMap.json')

OTR2_BEGINNING.to_json(JSON_OUT)

# KlystronDataMap

In [ ]:
from lcls_live.datamaps.klystron import KlystronDataMap, klystron_pvinfo, existing_LCLS_klystrons_sector_station

In [ ]:
# Get a sector, station that exists
existing_LCLS_klystrons_sector_station

In [ ]:
# This will return a flat dict of info
klystron_pvinfo(30, 6)

In [ ]:
?KlystronDataMap

In [ ]:
# This makes an object
KlystronDataMap(**klystron_pvinfo(21, 6))

In [ ]:
k = KlystronDataMap(**klystron_pvinfo(21, 6))

# These are the PV names needed to mapping data
k.pvlist


In [ ]:
# This will extract those and produce useful information
k.evaluate(PVDATA)

In [ ]:
# Actual inputs for a simulation
k.as_bmad(PVDATA)

In [ ]:
# A complete JSON string for serialization
k.to_json()

In [ ]:
# Make a large list

KLYSTRON_DATAMAPS = []
for sector, station in existing_LCLS_klystrons_sector_station:
    print(sector, station)    
    info = klystron_pvinfo(sector, station)
    k = KlystronDataMap(**info)

    KLYSTRON_DATAMAPS.append(k)

In [ ]:
# Check that our data is sufficient 

for k in KLYSTRON_DATAMAPS:
    for pv in k.pvlist:
        if pv not in PVDATA:
            print(k.name, pv)

In [ ]:
s = KLYSTRON_DATAMAPS[0].to_json()
s

In [ ]:
KlystronDataMap.from_json(s)

## Export all to JSON

In [ ]:

for i, identifier in enumerate(existing_LCLS_klystrons_sector_station):
    sector = identifier[0]
    station = identifier[1]

    JSONFILE = os.path.join(data_dir, f'cu/sector{sector}_station{station}_klystron_datamap.json')

    KLYSTRON_DATAMAPS[i].to_json(file=JSONFILE)

In [ ]:
# Export to file and reload
KLYSTRON_DATAMAPS2 = []
for i, identifier in enumerate(existing_LCLS_klystrons_sector_station):
    sector = identifier[0]
    station = identifier[1]
    
    JSONFILE = os.path.join(data_dir, f'cu/sector{sector}_station{station}_klystron_datamap.json')
    KLYSTRON_DATAMAPS2.append(KlystronDataMap.from_json(JSONFILE))



KLYSTRON_DATAMAPS2[0].pvlist